In [1]:
# ruff: noqa
import os

os.sys.path.append("..")
from feedback_grape.fgrape import optimize_pulse_with_feedback
from feedback_grape.utils.operators import (
    sigmap,
    sigmam,
    create,
    destroy,
    identity,
    cosm,
    sinm,
)
from feedback_grape.utils.states import basis, fock
from feedback_grape.utils.tensor import tensor
import jax.numpy as jnp
from jax.scipy.linalg import expm

## defining parameterized operations that are repeated num_time_steps times

In [2]:
N_cav = 5

In [3]:
def qubit_unitary(alpha):
    """
    TODO: see if alpha, can be sth elser other than scalar, and if the algo understands this
    see if there can be multiple params like alpha and beta input
    """
    return expm(
        -1j
        * (
            alpha * tensor(identity(N_cav), sigmap())
            + alpha.conjugate() * tensor(identity(N_cav), sigmam())
        )
        / 2
    )

In [4]:
def qubit_cavity_unitary(beta):
    return expm(
        -1j
        * (
            beta
            * (
                tensor(destroy(N_cav), identity(2))
                @ tensor(identity(N_cav), sigmap())
            )
            + beta.conjugate()
            * (
                tensor(create(N_cav), identity(2))
                @ tensor(identity(N_cav), sigmam())
            )
        )
        / 2
    )

In [5]:
alpha = 0.1 + 0.1j
beta = 0.1 + 0.1j
Uq = qubit_unitary(alpha)
Uqc = qubit_cavity_unitary(beta)
print(
    "Uq unitary check:",
    jnp.allclose(Uq.conj().T @ Uq, jnp.eye(Uq.shape[0]), atol=1e-7),
)
print(
    "Uqc unitary check:",
    jnp.allclose(Uqc.conj().T @ Uqc, jnp.eye(Uqc.shape[0]), atol=1e-7),
)

Uq unitary check: True
Uqc unitary check: True


In [6]:
qubit_unitary(0.1)

Array([[0.99875026+0.j        , 0.        -0.04997917j,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        -0.04997917j, 0.99875026+0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.99875026+0.j        , 0.        -0.04997917j,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.        -0.04997917j, 0.99875026+0.j        ,
        0.        +0.j        , 0.        +0.

In [7]:
from feedback_grape.utils.operators import create, destroy


def povm_measure_operator(measurement_outcome, gamma, delta):
    """
    POVM for the measurement of the cavity state.
    returns Mm ( NOT the POVM element Em = Mm_dag @ Mm ), given measurement_outcome m, gamma and delta
    """
    number_operator = tensor(create(N_cav) @ destroy(N_cav), identity(2))
    angle = (gamma * number_operator) + delta / 2
    meas_op = jnp.where(
        measurement_outcome == 1,
        cosm(angle),
        sinm(angle),
    )
    return meas_op

### defining initial (thermal) state

In [8]:
# initial state is a thermal state coupled to a qubit in the ground state?
n_average = 1
# natural logarithm
beta = jnp.log((1 / n_average) + 1)
diags = jnp.exp(-beta * jnp.arange(N_cav))
normalized_diags = diags / jnp.sum(diags, axis=0)
rho_cav = jnp.diag(normalized_diags)

In [9]:
rho_cav.shape

(5, 5)

In [10]:
rho0 = tensor(rho_cav, basis(2, 0) @ basis(2, 0).conj().T)

In [11]:
from feedback_grape.utils.povm import (
    _probability_of_a_measurement_outcome_given_a_certain_state,
)

_probability_of_a_measurement_outcome_given_a_certain_state(
    rho0, 1, povm_measure_operator, [0.1, -3 * jnp.pi / 2]
)

Array(0.88500904, dtype=float64)

### defining target state

In [12]:
psi_target = tensor(
    (fock(N_cav, 1) + fock(N_cav, 2) + fock(N_cav, 3)) / jnp.sqrt(3), basis(2)
)

rho_target = psi_target @ psi_target.conj().T
rho_target.shape

(10, 10)

In [13]:
from feedback_grape.utils.fidelity import fidelity

print(fidelity(U_final=rho0, C_target=rho_target, type="density"))

0.38799179683158536


### initialize random params

In [20]:
num_time_steps = 5
num_of_iterations = 1000
learning_rate = 0.005
# avg_photon_numer = 2 When testing kitten state


initial_params = {
    "POVM": [jnp.pi / 3, jnp.pi / 3],
    "U_q": [jnp.pi / 3],
    "U_qc": [jnp.pi / 3],
}


result = optimize_pulse_with_feedback(
    U_0=rho0,
    C_target=rho_target,
    parameterized_gates=[
        povm_measure_operator,
        qubit_unitary,
        qubit_cavity_unitary,
    ],
    measurement_indices=[0],
    initial_params=initial_params,
    num_time_steps=num_time_steps,
    mode="nn",
    goal="fidelity",
    optimizer="adam",
    max_iter=num_of_iterations,
    convergence_threshold=1e-20,
    learning_rate=learning_rate,
    type="density",
    batch_size=10,
)

Iteration 0, Loss: -0.049948
Iteration 10, Loss: 0.026424
Iteration 20, Loss: 0.161914
Iteration 30, Loss: 0.305665
Iteration 40, Loss: 0.377965
Iteration 50, Loss: 0.554871
Iteration 60, Loss: 0.548468
Iteration 70, Loss: 0.836883
Iteration 80, Loss: 0.598489
Iteration 90, Loss: 0.376973
Iteration 100, Loss: 0.530314
Iteration 110, Loss: 0.845635
Iteration 120, Loss: 0.770821
Iteration 130, Loss: 0.509334
Iteration 140, Loss: 0.637267
Iteration 150, Loss: 0.614934
Iteration 160, Loss: 0.816835
Iteration 170, Loss: 0.743203
Iteration 180, Loss: 0.935742
Iteration 190, Loss: 0.701891
Iteration 200, Loss: 0.628128
Iteration 210, Loss: 0.608445
Iteration 220, Loss: 0.571223
Iteration 230, Loss: 0.980320
Iteration 240, Loss: 0.848794
Iteration 250, Loss: 0.612082
Iteration 260, Loss: 0.604392
Iteration 270, Loss: 0.923348
Iteration 280, Loss: 0.640602
Iteration 290, Loss: 0.845768
Iteration 300, Loss: 0.912825
Iteration 310, Loss: 0.665470
Iteration 320, Loss: 0.846028
Iteration 330, Loss:

In [21]:
result

FgResult(optimized_rnn_parameters={'params': {'Dense_0': {'bias': Array([ 0.01531316,  0.05717081, -0.02392245, -0.2593045 ], dtype=float32), 'kernel': Array([[-0.39788583,  0.06039045, -0.2952888 , -0.29677656],
       [ 0.18346785,  0.15980601,  0.20220226, -0.00185075],
       [ 0.09205894, -0.00892506, -0.12275019,  0.10632998],
       [-0.07462677,  0.08972517,  0.1679145 , -0.10056113],
       [-0.3236614 , -0.1674962 , -0.4289308 ,  0.3116239 ],
       [-0.20334236, -0.09261465, -0.5993085 ,  0.19635658],
       [-0.31933692, -0.04006135, -0.02404811, -0.26392272],
       [ 0.24105722, -0.05003396,  0.38340464, -0.27553195],
       [-0.00895164,  0.08919179, -0.018963  ,  0.05774451],
       [-0.01852399, -0.02148866,  0.11890415,  0.5106456 ],
       [-0.02414727, -0.05067492, -0.04992865,  0.3055418 ],
       [ 0.17390917, -0.05259488,  0.41965505, -0.42720944],
       [ 0.13050823, -0.18419464, -0.10595518,  0.36994585],
       [ 0.24742496,  0.16453551,  0.34084228, -0.15596

In [22]:
print(result.final_purity)

None


In [23]:
print(result.final_fidelity)

0.6007848728984048


In [24]:
result.returned_params

[[Array([[1.04719755, 1.04719755],
         [1.04719755, 1.04719755],
         [1.04719755, 1.04719755],
         [1.04719755, 1.04719755],
         [1.04719755, 1.04719755],
         [1.04719755, 1.04719755],
         [1.04719755, 1.04719755],
         [1.04719755, 1.04719755],
         [1.04719755, 1.04719755],
         [1.04719755, 1.04719755]], dtype=float64),
  Array([[-0.30647609],
         [ 0.27335773],
         [ 0.27335773],
         [ 0.27335773],
         [-0.30647609],
         [-0.30647609],
         [ 0.27335773],
         [-0.30647609],
         [ 0.27335773],
         [ 0.27335773]], dtype=float64),
  Array([[-0.15208419],
         [-0.78192191],
         [-0.78192191],
         [-0.78192191],
         [-0.15208419],
         [-0.15208419],
         [-0.78192191],
         [-0.15208419],
         [-0.78192191],
         [-0.78192191]], dtype=float64)],
 [Array([[-0.25106854,  0.05447266],
         [ 0.06318987,  0.14913964],
         [ 0.06318987,  0.14913964],
       

In [19]:
from feedback_grape.utils.povm import povm
from feedback_grape.fgrape import apply_gate
from feedback_grape.utils.purity import purity
import jax

time_steps = 5

rho = rho0
print(
    "initial purity:",
    fidelity(U_final=rho, C_target=rho_target, type="density"),
)
time_step_keys = jax.random.split(jax.random.PRNGKey(2), time_steps)
print("time step keys:", time_step_keys.shape)
for i in range(time_steps):
    params = result.returned_params[i][0]
    rho, _, _ = povm(rho, povm_measure_operator, params[0], time_step_keys[i])
    rho = apply_gate(
        rho,
        qubit_unitary,
        params[1],
        type="density",
    )
    rho = apply_gate(
        rho,
        qubit_cavity_unitary,
        params[2],
        type="density",
    )
    print(
        f"fid of rho after time step {i}",
        fidelity(U_final=rho, C_target=rho_target, type="density"),
    )
final_rho_cav = rho

initial purity: 0.38799179683158536
time step keys: (5, 2)


TypeError: qubit_unitary() takes 1 positional argument but 2 were given

In [ ]:
print(result.iterations)

1000


In [ ]:
print(result.returned_params[1])

[[-0.024289291206994038, -0.6440354417453446], [-4.089295561711708], [-3.4806313321334637]]


In [ ]:
import jax

F = []
F_1_0 = jnp.array([jnp.pi / 2, 0])
key = jax.random.PRNGKey(0)
appo = jax.random.uniform(key, shape=(2**1, 4)) * jnp.pi
F.append(appo)
F

[Array([[1.31462179, 0.67951221, 3.03264681, 1.80484666],
        [1.67203883, 1.11496751, 2.77405451, 1.98724708]], dtype=float64)]

In [ ]:
variables = [F] + [F_1_0]

In [ ]:
variables

[[Array([[1.31462179, 0.67951221, 3.03264681, 1.80484666],
         [1.67203883, 1.11496751, 2.77405451, 1.98724708]], dtype=float64)],
 Array([1.57079633, 0.        ], dtype=float64)]

In [ ]:
def prepare_parameters_from_dict(params_dict):
    """
    Convert a nested dictionary of parameters to a flat list and record shapes.

    Args:
        params_dict: Nested dictionary of parameters.

    Returns:
        tuple: Flattened parameters list and list of shapes.
    """
    flat_params = []
    param_shapes = []

    # returns a flat list of the leaves
    def flatten_dict(d):
        result = []
        for key, value in d.items():
            if isinstance(value, dict):
                result.extend(flatten_dict(value))
            else:
                result.append(value)
        return result

    # flatten each top-level gate
    for gate_name, gate_params in params_dict.items():
        if isinstance(gate_params, dict):
            # Extract parameters for this gate
            gate_flat_params = jnp.array(flatten_dict(gate_params))
        else:
            # If already a flat array
            gate_flat_params = jnp.array(gate_params)
        # this is checking if use can enter sth like {'gate1': 1} instead of {"gate1": {"param1": 1}}
        # if not (isinstance(gate_flat_params, list)):
        #     flat_params.append([gate_flat_params])
        #     param_shapes.append(1)
        # else:
        #     flat_params.append(gate_flat_params)
        #     param_shapes.append(len(gate_flat_params))
        flat_params.append(gate_flat_params)
        param_shapes.append(gate_flat_params.shape)
    return flat_params, param_shapes

In [ ]:
import numpy as np


def reshape_params(param_shapes, rnn_flattened_params):
    """
    Reshape the parameters for the gates.
    """
    # Reshape the flattened parameters from RNN output according
    # to each gate corressponding params
    reshaped_params = []
    param_idx = 0
    for shape in param_shapes:
        num_params = int(np.prod(shape))
        # rnn outputs a flat list, this takes each and assigns according to the shape
        gate_params = rnn_flattened_params[
            param_idx : param_idx + num_params
        ].reshape(shape)
        reshaped_params.append(gate_params)
        param_idx += num_params

    new_params = reshaped_params
    return new_params

In [ ]:
import os

os.sys.path.append("..")
import jax
import jax.numpy as jnp
from feedback_grape.fgrape import prepare_parameters_from_dict, reshape_params

initial_params = {
    "POVM": [jnp.pi / 3, jnp.pi / 3],
    "U_q": [jnp.pi / 3],
    "U_qc": [jnp.pi / 3],
}

flat_params, param_shapes = prepare_parameters_from_dict(initial_params)
num_of_columns = 4
num_of_sub_lists = 3
F = []


def construct_ragged_row(num_of_rows):
    res = []
    for i in range(num_of_rows):
        flattened = jax.random.uniform(
            jax.random.PRNGKey(0 + i),
            shape=(num_of_columns,),
            minval=-jnp.pi,
            maxval=jnp.pi,
        )
        res.append(flattened)
    return res


for i in range(1, num_of_sub_lists + 1):
    F.append(construct_ragged_row(num_of_rows=2**i))

for i in range(len(F)):
    print("length of F[{}]: {}".format(i, len(F[i])))

print("############")
min_num_of_rows = 2 ** len(F)
for i in range(len(F)):
    if len(F[i]) < min_num_of_rows:
        zeros_arrays = [
            jnp.zeros((num_of_columns,), dtype=jnp.float32)
            for _ in range(min_num_of_rows - len(F[i]))
        ]
        F[i] = F[i] + zeros_arrays

print("############")
from pprint import pprint

for i in range(len(F[0])):
    print("length of F[{}]: {}".format(i, len(F[0][i])))
    pprint(F[0][i])

length of F[0]: 2
length of F[1]: 4
length of F[2]: 8
############
############
length of F[0]: 4
Array([-0.51234908, -1.78256823,  2.92370097,  0.46810066], dtype=float64)
length of F[1]: 4
Array([-2.39923276, -0.29054238,  0.43072842,  2.07843927], dtype=float64)
length of F[2]: 4
Array([0., 0., 0., 0.], dtype=float32)
length of F[3]: 4
Array([0., 0., 0., 0.], dtype=float32)
length of F[4]: 4
Array([0., 0., 0., 0.], dtype=float32)
length of F[5]: 4
Array([0., 0., 0., 0.], dtype=float32)
length of F[6]: 4
Array([0., 0., 0., 0.], dtype=float32)
length of F[7]: 4
Array([0., 0., 0., 0.], dtype=float32)


In [ ]:
print(F)

[[Array([-0.51234908, -1.78256823,  2.92370097,  0.46810066], dtype=float64), Array([-2.39923276, -0.29054238,  0.43072842,  2.07843927], dtype=float64), Array([0., 0., 0., 0.], dtype=float32), Array([0., 0., 0., 0.], dtype=float32), Array([0., 0., 0., 0.], dtype=float32), Array([0., 0., 0., 0.], dtype=float32), Array([0., 0., 0., 0.], dtype=float32), Array([0., 0., 0., 0.], dtype=float32)], [Array([-0.51234908, -1.78256823,  2.92370097,  0.46810066], dtype=float64), Array([-2.39923276, -0.29054238,  0.43072842,  2.07843927], dtype=float64), Array([-0.4773859 , -2.20922398,  1.04214091, -0.38718841], dtype=float64), Array([ 2.2958313 , -0.29402536, -1.6428162 , -1.85833117], dtype=float64), Array([0., 0., 0., 0.], dtype=float32), Array([0., 0., 0., 0.], dtype=float32), Array([0., 0., 0., 0.], dtype=float32), Array([0., 0., 0., 0.], dtype=float32)], [Array([-0.51234908, -1.78256823,  2.92370097,  0.46810066], dtype=float64), Array([-2.39923276, -0.29054238,  0.43072842,  2.07843927], dt

In [ ]:
def convert_to_index(measurement_history):
    # Convert measurement history from [1, -1, ...] to [0, 1, ...] and then to an integer index
    binary_history = jnp.where(jnp.array(measurement_history) == 1, 0, 1)
    print(f"binary_history: {binary_history}")
    # Convert binary list to integer index (e.g., [0,1] -> 1)
    # Reverse the binary_history to operate from last element backwards
    reversed_binary = binary_history[::-1]
    int_index = sum(
        (2**i) * reversed_binary[i] for i in range(len(reversed_binary))
    )
    return int_index


def extract_from_lut(lut, measurement_history):
    """
    Extract parameters from the lookup table based on the measurement history.

    Args:
        lut: Lookup table for parameters.
        measurement_history: History of measurements.
        time_step: Current time step.

    Returns:
        Extracted parameters.
    """
    sub_array_idx = len(measurement_history) - 1
    print(f"sub_array_idx: {sub_array_idx}")
    sub_array_param_idx = convert_to_index(measurement_history)
    print(f"sub_array_param_idx: {sub_array_param_idx}")
    return lut[sub_array_idx][sub_array_param_idx]


extracted_lut_params = extract_from_lut(F, [1, -1])

sub_array_idx: 1
binary_history: [0 1]
sub_array_param_idx: 1


In [ ]:
print(extracted_lut_params)

[-2.39923276 -0.29054238  0.43072842  2.07843927]


In [ ]:
import tensorflow as tf

print(tf.random.uniform(shape=[1]))

tf.Tensor([0.43207276], shape=(1,), dtype=float32)


In [ ]:
import jax

jax.random.uniform(
    jax.random.PRNGKey(0),
    shape=(1,),
)

Array([0.41845711], dtype=float64)